# Components of Influence V.S. Euclidean

A training point with the most positive influence to a test point can be intuitively perceived as its closest neighbor in Euclidean space, and vice versa (Ribeiro et al. (2016)). However, using the Euclidean distance is not the optimal method. In this session we are going to prove that influence function is more accurate in capturing the training effect. To simplify the problem, we use MINST two case data set and logistic regression model here.





Assuming all points have the same norm, the Euclidean distance between two points is 

\begin{aligned}
 D_{Euc} = x \cdot x_{test} 
\end{aligned}

As for the closed-form influnce function in logistic regression, let $p(y|x)=\sigma (y\theta^Tx)$, with y belonging to {-1, 1} and $\sigma(t)=\frac{1}{1+exp(-t)}$. For any training point $z=(x,y)$, $L(z,\theta)=log(1+exp(-y\theta^Tx))$, $\nabla_{\theta} L(z,\theta)=-\sigma(-y\theta^Tx)yx$, and $H_\theta=-\frac{1}{n}\sum_{i=1}^{n}\sigma(\theta^Tx_i)\sigma(-\theta^Tx_i)x_ix_i^T$. For MINST dataset specifically, since there are many 0 in $x_I$, the Hessian matrix is very likely to be singular. Therefore, we add a small lambda term on its diagonal to make new $H_\theta=H_\theta+\lambda I$. From the previous loss influence function, we have:

\begin{aligned}
\mathcal{I}_{loss}({z}, {z_{test}})=-y_{test}y \cdot \sigma(-y_{test}\theta^Tx_{test}) \cdot \sigma(-y\theta^Tx) \cdot x^T_{test}H^{-1}_{\hat{\theta}}x 
\end{aligned}

We notice that there are two major differences between influence function and Euclidean distance. The first is the $\sigma(-y\theta^Tx)$, which gives more influence/leverage to the outlier training points (high training loss). The second is the Hessian inverse matrix $H^{-1}_{\hat{\theta}}$, which "measures the resistance of the other training points to the removal of z" (Koh et al. 2017). It means that the influence of z will be bigger if other training points have little impact on z. If we take both components out from the influence function, what is left is the scaled Euclidean distance $-y_{test}y \cdot \sigma(-y_{test}\theta^Tx_{test}) \cdot x^T_{test}x$. In figure 1, we compare the influence function with and without each components.  

In [4]:
from IPython.display import Image
Image("[Fig1]component influence.png")

_Figure 1._ **Components of influence**. This graph fits the pattern in the original. We plot the completed negative influence loss function against variants that are missing different terms. Yellow dots are the training data with the same label as the test data; purple dots are the oppisite labels.

__Left:__ Without the train loss term, we overestimate the influence of many training points: the points near the y=0 line should have I_up,loss close to 0, but instead have high influence (either positive or negative) when we remove the train loss term.
__Mid:__ Without Hessian, all yellow points are benefitial to the prediction (removing each point will increase the loss) while all purple points are harmful (removing each point will decrease the loss). But this is not true since some same-label training data can have negative impact to the model (see the image on the right: both have lable 7, but the harmful training image does no good to the model). 
__Right:__ Without training loss or Hessian, what is left is the scaled Euclidean inner product. This is similar to the mid graph but spread more widely. Therefore, we can conclude that Euclidean distance is not as accurate as the influence function because it cannot absort positive impact from wrong-label training data, or vice versa.

# Reference

Ribeiro, M. T., Singh, S., and Guestrin, C. "why should
I trust you?": Explaining the predictions of any classifier.
In _International Conference on Knowledge Discovery
and Data Mining (KDD)_, 2016.

Koh, Pang Wei, and Percy Liang. "Understanding black-box predictions via influence functions." arXiv preprint arXiv:1703.04730 (2017).